In [13]:
# Create file path variable

fhvTaxisFilePath = 'abfss://taxidata@pstaxisdatalake.dfs.core.windows.net/FhvTaxis*.csv'

StatementMeta(PSSparkPool, 1, 14, Finished, Available)

In [3]:
# Read FHV Taxis data

fhvTaxiTripDataDF = (
                        spark
                            .read

                            .option("header", "true")
                            .option("inferSchema", "true")

                            .csv(fhvTaxisFilePath)
                    )

fhvTaxiTripDataDF.printSchema

StatementMeta(PSSparkPool, 1, 3, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: int]>

In [4]:
# Display FHV Taxis data

display(
    fhvTaxiTripDataDF.limit(1000)
)

StatementMeta(PSSparkPool, 1, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, b3b3b7ea-e92d-4043-8ee5-b2ece2a8d80a)

In [5]:
#Display summary of FHV Taxis

display(
    fhvTaxiTripDataDF,
    summary=True
)

StatementMeta(PSSparkPool, 1, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 06ac6288-c666-4840-a25d-ccb4928edeee)

In [6]:
# Select only limited columns

fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF
                            .select(
                                    "hvfhs_license_num",
                                    "dispatching_base_num",
                                    "Pickup_DateTime", 
                                    "DropOff_DateTime", 
                                    "PUlocationID", 
                                    "DOlocationID"                                    
                                )
                    )

fhvTaxiTripDataDF.printSchema

StatementMeta(PSSparkPool, 1, 7, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[hvfhs_license_num: string, dispatching_base_num: string, Pickup_DateTime: timestamp, DropOff_DateTime: timestamp, PUlocationID: int, DOlocationID: int]>

In [7]:
# Rename the columns

fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF
                            .withColumnRenamed("hvfhs_license_num", "CompanyLicenseId")
                            .withColumnRenamed("dispatching_base_num", "BaseLicenseId")
                            .withColumnRenamed("Pickup_DateTime", "PickupTime")
                            .withColumnRenamed("DropOff_DateTime", "DropTime")
                            .withColumnRenamed("PUlocationID", "PickupLocationId")
                            .withColumnRenamed("DOlocationID", "DropLocationId")                            
                    )

fhvTaxiTripDataDF.printSchema

StatementMeta(PSSparkPool, 1, 8, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[CompanyLicenseId: string, BaseLicenseId: string, PickupTime: timestamp, DropTime: timestamp, PickupLocationId: int, DropLocationId: int]>

In [8]:
from pyspark.sql.functions import *

# Create derived columns for year, month and day
fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF
                            .withColumn("TripYear", year(col("PickupTime")))
                            .withColumn("TripMonth", month(col("PickupTime")))
                            .withColumn("TripDay", dayofmonth(col("PickupTime")))
                    )

fhvTaxiTripDataDF.printSchema

StatementMeta(PSSparkPool, 1, 9, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[CompanyLicenseId: string, BaseLicenseId: string, PickupTime: timestamp, DropTime: timestamp, PickupLocationId: int, DropLocationId: int, TripYear: int, TripMonth: int, TripDay: int]>

In [9]:
# Filter inaccurate data

fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF
                            .where("PickupTime >= '2019-11-01' AND PickupTime < '2019-12-01'")
                    )


display(fhvTaxiTripDataDF.limit(100))

StatementMeta(PSSparkPool, 1, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7dba858c-c8ab-4607-8175-fc9bd75f35e2)

In [12]:
# Create file path variable

fhvBasesFilePath = 'abfss://taxidata@pstaxisdatalake.dfs.core.windows.net/FhvBases.json'

StatementMeta(PSSparkPool, 1, 13, Finished, Available)

In [14]:
# Read FHV Bases json file

fhvBasesDF = (
                spark
                  .read
                  .option("multiline", "true")
                  .json(fhvBasesFilePath)
             )

display(fhvBasesDF)

StatementMeta(PSSparkPool, 1, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8d2258f1-796c-43f0-b357-745bfc450169)

In [15]:
from pyspark.sql.functions import *

# Flatten FHV Bases data

fhvBasesFlatDF = (
                    fhvBasesDF
                        .select(
                                    col("License Number").alias("BaseLicenseId"),
                                    col("Type of Base").alias("BaseType"),

                                    col("Address.Building").alias("AddressBuilding"),
                                    col("Address.Street").alias("AddressStreet"),
                                    col("Address.City").alias("AddressCity"),
                                    col("Address.State").alias("AddressState"),
                                    col("Address.PostCode").alias("AddressPostalCode")
                               )
                )

display(fhvBasesFlatDF)

StatementMeta(PSSparkPool, 1, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, f8352cc3-98c8-4bae-81e5-fe42c2612521)

In [16]:
# Create a dataframe joining FHV trip data with bases

fhvTaxiTripDataWithBasesDF = (
                                fhvTaxiTripDataDF
                                     .join(fhvBasesFlatDF,                                               
                                               "BaseLicenseId",
                                                "inner"
                                          )
                             )

display(fhvTaxiTripDataWithBasesDF)

StatementMeta(PSSparkPool, 1, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 08b50e53-230e-466c-817f-076f7dfb9767)

In [ ]:
from pyspark.sql.functions import *

fhvTaxiTripDataDF = (
                        spark
                            .read
                            .option("header", "true")
                            .option("inferSchema", "true")
                            .csv(fhvTaxisFilePath)
                    )

fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF

                            # Select limited columns
                            .select(
                                    "hvfhs_license_num",
                                    "dispatching_base_num",
                                    "Pickup_DateTime", 
                                    "DropOff_DateTime", 
                                    "PUlocationID", 
                                    "DOlocationID"                                    
                                )

                            #Rename the columns
                            .withColumnRenamed("hvfhs_license_num", "CompanyLicenseId")
                            .withColumnRenamed("dispatching_base_num", "BaseLicenseId")
                            .withColumnRenamed("Pickup_DateTime", "PickupTime")
                            .withColumnRenamed("DropOff_DateTime", "DropTime")
                            .withColumnRenamed("PUlocationID", "PickupLocationId")
                            .withColumnRenamed("DOlocationID", "DropLocationId")

                            # Create derived columns for year, month and day
                            .withColumn("TripYear", year(col("PickupTime")))
                            .withColumn("TripMonth", month(col("PickupTime")))
                            .withColumn("TripDay", dayofmonth(col("PickupTime")))

                            # Filter records based on PickupTime
                            .where("PickupTime >= '2019-11-01' AND PickupTime < '2019-12-01'")
                    )


# Flatten FHV Bases data
fhvBasesFlatDF = (
                    fhvBasesDF
                        .select(
                                    col("License Number").alias("BaseLicenseId"),
                                    col("Type of Base").alias("BaseType"),

                                    col("Address.Building").alias("AddressBuilding"),
                                    col("Address.Street").alias("AddressStreet"),
                                    col("Address.City").alias("AddressCity"),
                                    col("Address.State").alias("AddressState"),
                                    col("Address.PostCode").alias("AddressPostalCode")
                               )
                )

# Create a dataframe joining FHV trip data with bases
fhvTaxiTripDataWithBasesDF = (
                                fhvTaxiTripDataDF
                                     .join(fhvBasesFlatDF,                                               
                                               "BaseLicenseId",
                                                "inner"
                                          )
                             )

display(fhvTaxiTripDataWithBasesDF)

In [17]:
%%sql

CREATE DATABASE FhvWarehouse

StatementMeta(PSSparkPool, 1, 18, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [19]:
(
    fhvTaxiTripDataDF
        .write
        .partitionBy("TripYear", "TripMonth", "TripDay")
        .mode("overwrite")
        .parquet("abfss://taxioutput@pstaxisdatalake.dfs.core.windows.net/Facts/FhvTaxis.parquet")
)

StatementMeta(PSSparkPool, 1, 20, Finished, Available)

In [20]:
(
    fhvTaxiTripDataDF
        .write
        .partitionBy("TripYear", "TripMonth", "TripDay")
        .mode("overwrite")        
        .saveAsTable("FhvWarehouse.FHVTripsManaged")
)

StatementMeta(PSSparkPool, 1, 21, Finished, Available)

In [21]:
(
    fhvTaxiTripDataDF
        .write
        .partitionBy("TripYear", "TripMonth", "TripDay")
        .mode("overwrite")
        .option("path", "abfss://taxioutput@pstaxisdatalake.dfs.core.windows.net/Facts/FhvTaxis.parquet")
        .saveAsTable("FhvWarehouse.FHVTrips")
)

StatementMeta(PSSparkPool, 1, 22, Finished, Available)

In [22]:
(
    fhvBasesFlatDF
        .write        
        .mode("overwrite")
        .option("path", "abfss://taxioutput@pstaxisdatalake.dfs.core.windows.net/Dimensions/FhvBases.parquet")
        .saveAsTable("FhvWarehouse.FHVBases")
)

StatementMeta(PSSparkPool, 1, 23, Finished, Available)